In [ ]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
import pandas as pd 
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib

from __future__ import print_function


from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/power_roimask_4bi.nii.gz'
#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/w1_milkshake_partial.csv'
#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/w1_milkshake_partial.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
y = behavioral["Label"]


#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['LF_HS_receipt', "h20_receipt"])
y = y[condition_mask]

#confirm we have the # of condtions needed
print(y.unique())


masker = NiftiMasker(mask_img=imag_mask,
                     standardize=True, memory="nilearn_cache", memory_level=2)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]



# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

Automatically created module for IPython interactive environment  
['h20_receipt' 'LF_HS_receipt']  
# Tuning hyper-parameters for precision  

Best parameters set found on development set:  

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}  

Grid scores on development set:  


0.486 (+/-0.051) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}  
0.466 (+/-0.048) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.607 (+/-0.023) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}  
0.478 (+/-0.046) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.623 (+/-0.016) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}  
0.588 (+/-0.035) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.623 (+/-0.016) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}  
0.604 (+/-0.040) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.547 (+/-0.038) for {'C': 1, 'kernel': 'linear'}  
0.547 (+/-0.038) for {'C': 10, 'kernel': 'linear'}    
0.547 (+/-0.038) for {'C': 100, 'kernel': 'linear'}   
0.547 (+/-0.038) for {'C': 1000, 'kernel': 'linear'}  
    
Detailed classification report:  
  
The model is trained on the full development set.  
The scores are computed on the full evaluation set.    
  
                    precision    recall  f1-score   support  

    LF_HS_receipt      0.65      0.66      0.66      1455
    h20_receipt        0.65      0.64      0.65      1437

    micro avg          0.65      0.65      0.65      2892
    macro avg          0.65      0.65      0.65      2892
    weighted avg       0.65      0.65      0.65      2892


# Tuning hyper-parameters for recall  
  
Best parameters set found on development set:  
  
{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}  
  
Grid scores on development set:  
  
0.486 (+/-0.049) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}  
0.469 (+/-0.043) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.607 (+/-0.023) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}  
0.479 (+/-0.045) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.623 (+/-0.017) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}  
0.588 (+/-0.035) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.623 (+/-0.017) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}  
0.604 (+/-0.040) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}  
0.547 (+/-0.038) for {'C': 1, 'kernel': 'linear'}  
0.547 (+/-0.038) for {'C': 10, 'kernel': 'linear'}  
0.547 (+/-0.038) for {'C': 100, 'kernel': 'linear'}  
0.547 (+/-0.038) for {'C': 1000, 'kernel': 'linear'}  
  
Detailed classification report:  

The model is trained on the full development set.  
The scores are computed on the full evaluation set.  

                   precision    recall  f1-score   support

    LF_HS_receipt      0.65      0.66      0.66      1455  
    h20_receipt        0.65      0.64      0.65      1437  

    micro avg          0.65      0.65      0.65      2892
    macro avg          0.65      0.65      0.65      2892
    weighted avg       0.65      0.65      0.65      2892